# Mask R-CNN Training on the Shapes Dataset


This notebook explains how to train Mask R-CNN on your own dataset. 

To keep things simple we use a synthetic dataset of shapes (squares, triangles, and circles) and train the model on it. 

The code for the *Shapes* dataset is included below. It generates images on the fly, so it doesn't require downloading any data. And it can generate images of any size, so we pick a small image size for faster training. 

The other benefit of using the Shapes dataset is that the shapes are fairly simple, so training converges quickly. You'd still need a GPU, though, because the network backbone is a Resnet101, which would be too slow to train on a CPU. On a GPU, you can start to get okay results in a few minutes, and good results in a couple of hours or less.


In [ ]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt

from config import Config
import common
import model as modellib
import visualize
from visualize import tensor_summary, display_images

%matplotlib inline 


ROOT_DIR = os.getcwd()
MODEL_DIR = ROOT_DIR  # todo

## Configurations

In [ ]:
class ShapesConfig(Config):
    """Configuration for training on the toy shapes dataset.
    Derives from the base Config class and overrides values specific
    to the toy shapes dataset.
    """
    # Give the configuration a recognizable name
    NAME = "SHAPES"

    # Train on 1 GPU and 8 images per GPU. We can put multiple images on each
    # GPU because the images are small. Batch size is 8 (GPUs * images/GPU).
    GPU_COUNT = 1
    IMAGES_PER_GPU = 8

    # Number of classes (including background)
    NUM_CLASSES = 1 + 3  # background + 3 shapes

    # Use small images for faster training. Set the limits of the small side
    # the large side, and that determines the image shape.
    IMAGE_MIN_DIM = 128
    IMAGE_MAX_DIM = 128

    # Use smaller anchors because our image and objects are small
    RPN_ANCHOR_SCALES = (8, 16, 32, 64, 128)  # anchor side in pixels

    # Reduce training ROIs per image because the images are small and have
    # few objects. Aim to allow ROI sampling to pick 33% positive ROIs.
    TRAIN_ROIS_PER_IMAGE = 32

    # todo:
    # POST_NMS_ROIS_INFERENCE = 100
    # DETECTION_MIN_CONFIDENCE = 0.2
    LEARNING_RATE = 0.002
    STEPS_PER_EPOCH = 100

    
config = ShapesConfig()
config.print()

## Dataset

Create a synthetic dataset

Overrde the following methods:

* load_shapes()
* load_mask()
* external_url()


In [ ]:
class ShapesDataset(common.Dataset):
    """Generates the shapes synthetic dataset. The dataset consists of simple
    shapes (triangles, squares, circles) placed randomly on a blank surface.
    The images are generated on the fly. No file access required.
    """

    def load_shapes(self, count, height, width):
        """Generate the requested number of synthetic images.
        count: number of images to generate.
        height, width: the size of the generated images.
        """
        # Add classes
        self.add_class("shapes", 1, "square")
        self.add_class("shapes", 2, "circle")
        self.add_class("shapes", 3, "triangle")

        # Add images
        # Generate random specifications of images (i.e. color and
        # list of shapes sizes and locations). This is more compact than
        # actual images. Images are generated on the fly in load_image().
        for i in range(count):
            bg_color, shapes = self.random_image(height, width)
            self.add_image("shapes", image_id=i, path=None,
                           width=width, height=height,
                           bg_color=bg_color, shapes=shapes)

    def load_image(self, image_id):
        """Generate an image from the specs of the given image ID.
        Typically this function loads the image from a file, but
        in this case it generates the image on the fly from the
        specs in image_info.
        """
        info = self.image_info[image_id]
        bg_color = np.array(info['bg_color']).reshape([1, 1, 3])
        image = np.ones([info['height'], info['width'], 3], dtype=np.uint8)
        image = image * bg_color.astype(np.uint8)
        for shape, color, dims in info['shapes']:
            image = self.draw_shape(image, shape, dims, color)
        return image

    def load_mask(self, image_id):
        """Generate instance masks for shapes of the given image ID.
        """
        info = self.image_info[image_id]
        shapes = info['shapes']
        count = len(shapes)
        mask = np.zeros([info['height'], info['width'], count], dtype=np.uint8)
        for i, (shape, _, dims) in enumerate(info['shapes']):
            mask[:, :, i:i+1] = self.draw_shape(mask[:, :, i:i+1].copy(),
                                                shape, dims, 1)
        # Handle occlusions
        occlusion = np.logical_not(mask[:, :, -1]).astype(np.uint8)
        for i in range(count-2, -1, -1):
            mask[:, :, i] = mask[:, :, i] * occlusion
            occlusion = np.logical_and(occlusion, np.logical_not(mask[:, :, i])) # todo: .astype(np.uint8)
        # Map class names to class IDs.
        class_ids = np.array([self.class_names.index(s[0]) for s in shapes])
        return mask, class_ids

    def draw_shape(self, image, shape, dims, color):
        """Draws a shape from the given specs."""
        # Get the center x, y and the size s
        x, y, s = dims
        if shape == 'square':
            image = cv2.rectangle(image, (x-s, y-s), (x+s, y+s), color, -1)
        elif shape == "circle":
            image = cv2.circle(image, (x, y), s, color, -1)
        elif shape == "triangle":
            points = np.array([[(x, y-s),
                                (x-s/math.sin(math.radians(60)), y+s),
                                (x+s/math.sin(math.radians(60)), y+s),
                                ]], dtype=np.int32)
            image = cv2.fillPoly(image, points, color)
        return image

    def random_shape(self, height, width):
        """Generates specifications of a random shape that lies within
        the given height and width boundaries.
        Returns a tuple of three valus:
        * The shape name (square, circle, ...)
        * Shape color: a tuple of 3 values, RGB.
        * Shape dimensions: A tuple of values that define the shape size
                            and location. Differs per shape type.
        """
        # Shape
        shape = random.choice(["square", "circle", "triangle"])
        # Color
        color = tuple([random.randint(0, 255) for _ in range(3)])
        # Center x, y
        buffer = 20
        y = random.randint(buffer, height - buffer - 1)
        x = random.randint(buffer, width - buffer - 1)
        # Size
        s = random.randint(buffer, height//4)
        return shape, color, (x, y, s)

    def random_image(self, height, width):
        """Creates random specifications of an image with multiple shapes.
        Returns the background color of the image and a list of shape
        specifications that can be used to draw the image.
        """
        # Pick random background color
        bg_color = np.array([random.randint(0, 255) for _ in range(3)])
        # Generate a few random shapes and record their
        # bounding boxes
        shapes = []
        boxes = []
        N = random.randint(1, 4)
        for _ in range(N):
            shape, color, dims = self.random_shape(height, width)
            shapes.append((shape, color, dims))
            x, y, s = dims
            boxes.append([y-s, x-s, y+s, x+s])
        # Apply non-max suppression wit 0.3 threshold to avoid
        # shapes covering each other
        keep_ixs = common.non_max_suppression(np.array(boxes), np.arange(N), 0.3)
        shapes = [s for i, s in enumerate(shapes) if i in keep_ixs]
        return bg_color, shapes

In [ ]:
# Training dataset
dataset_train = ShapesDataset()
dataset_train.load_shapes(500, config.IMAGE_SHAPE[0], config.IMAGE_SHAPE[1])
dataset_train.prepare()

# Validation dataset
dataset_val = ShapesDataset()
dataset_val.load_shapes(50, config.IMAGE_SHAPE[0], config.IMAGE_SHAPE[1])
dataset_val.prepare()

In [ ]:
# Load and display random samples
image_ids = np.random.choice(dataset_train.image_ids, 4)
for image_id in image_ids:
    image = dataset_train.load_image(image_id)
    mask, class_ids = dataset_train.load_mask(image_id)
    visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names)

## Load Model

In [ ]:
# Load Weights Trained on MS COCO
TEST_MODEL = os.path.join(MODEL_DIR, "mask_rcnn_coco.h5")

In [ ]:
# Create model and load weights
model = modellib.MaskRCNN(mode="training", config=config)
# # model.load_weights(TEST_MODEL)
model.load_weights(TEST_MODEL, by_name=True,
                   exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", "mrcnn_bbox", "mrcnn_mask"])


## Training

In [ ]:
model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE, 
            epochs=1, 
            layers='heads')

In [ ]:
# Save weights
# Typically not needed because callbacks save after every epoch
temp_path = os.path.join(MODEL_DIR, "temp_weights.h5")
model.keras_model.save_weights(temp_path)


class InferenceConfig(ShapesConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    USE_MINI_MASK = False  # To get full masks from the data generator

inference_config = InferenceConfig()

# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", config=inference_config)
model.load_weights(temp_path, by_name=True)

# todo:
# model.load_weights(os.path.join(MODEL_DIR, "temp_weights_epoch22.h5"), by_name=True)

# TODO: Delete the temp file


## Detection

In [ ]:
image_id = random.choice(dataset_val.image_ids)
original_image, image_meta, gt_bbox, gt_mask =\
    modellib.load_image_gt(dataset_val, inference_config, 
                           image_id, use_mini_mask=False)

tensor_summary(original_image, "original_image")
tensor_summary(image_meta, "image_meta")
tensor_summary(gt_bbox, "gt_bbox")
tensor_summary(gt_mask, "gt_mask")

# display_images([image]+[mask[:,:,i] for i in range(min(mask.shape[-1], 7))])
visualize.display_instances(original_image, gt_bbox[:,:4], gt_mask, gt_bbox[:,4], 
                            dataset_train.class_names, figsize=(8, 8))

In [ ]:
original_shape = original_image.shape  # todo: necessary?

images = np.expand_dims(common.mold_image(original_image, inference_config), 0)
image_metas = np.expand_dims(image_meta, 0)
tensor_summary(images)

In [ ]:
# Run object detection
detections, mrcnn_class, mrcnn_bbox, mrcnn_mask, rois, \
rpn_class, rpn_bbox = model.keras_model.predict([images, image_metas], verbose=1)


In [ ]:
# Parse image_meta
_, _, window, _ = common.parse_image_meta(image_metas)
window

In [ ]:
# Process detections
final_rois, final_class_ids, final_scores, final_masks =\
    modellib.process_image_detections(detections[0], mrcnn_mask[0], 
                                      original_shape, window[0], config)

visualize.display_instances(original_image, 
                            final_rois, 
                            np.transpose(final_masks, [1, 2, 0]), 
                            final_class_ids, dataset_val.class_names,
                            final_scores, figsize=(8, 8))

## Evaluation

In [ ]:
image_ids = np.random.choice(dataset_val.image_ids, 10)
APs = []
for image_id in image_ids:
    # Load image
    original_image, image_meta, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset_val, inference_config, 
                               image_id, use_mini_mask=False)
    original_shape = original_image.shape  # todo: necessary?
    images = np.expand_dims(common.mold_image(original_image, inference_config), 0)
    image_metas = np.expand_dims(image_meta, 0)
    # Run object detection
    detections, mrcnn_class, mrcnn_bbox, mrcnn_mask, rois, \
    rpn_class, rpn_bbox = model.keras_model.predict([images, image_metas], verbose=1)
    # Process detections
    final_rois, final_class_ids, final_scores, final_masks =\
        modellib.process_image_detections(detections[0], mrcnn_mask[0], 
                                          original_shape, window[0], config)
    # Compute AP
    AP, precisions, recalls, overlaps =\
        common.compute_ap(gt_bbox[:,:4], gt_bbox[:,4],
                          detections[0, :, :4], detections[0, :, 4], detections[0,:,5])
    APs.append(AP)
    
print("mAP: ", np.mean(APs))


In [ ]:
visualize.plot_precision_recall(AP, precisions, recalls)